In [1]:
# Load Required Libraries
import os
import sys
import glob
import calendar

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import  mapping #shape, 
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask
# from contextlib import contextmanager

from osgeo import gdal #, osr, ogr


import matplotlib.pyplot as plt

In [2]:
# Load Functions defined in functions4stepFunction jupyter notebook

%run functions4stepFunction.ipynb

In [3]:
# Define Annual and Pernnial crops

allcrops = {'Barley':'Bar', 'Cassava':'Cas', 'Cotton':'Cot', 'Fodder':'Fod', 'Groundnuts':'Gro',
            'Maize':'Maz', 'Millet':'Mil', 'Oil palm':'Olp', 'Others annual':'Otha', 'Others perennial':'Othp',
            'Potatoes':'Pot', 'Pulses':'Pul', 'Rapeseed':'Rap', 'Rice':'Ric', 'Rye':'Rye', 'Sorghum':'Sor',
            'Soybeans':'Soy', 'Coffee':'Stm', 'Cocoa':'Stm', 'Sugar beet':'Sub', 'Sugar cane':'Sug', 'Sunflower':'Sun', 
            'Wheat':'Whe'}
Annual = ("Barley", "Cassava", "Groundnuts","Maize","Millet",
          "Potatoes","Pulses","Rapeseed","Rice1","Rice2",
          "Sorghum","Soybeans","Sunflower","Wheat1","Wheat2",
          "Cotton", "Fodder", "Others annual", "Rye", "Sugar beet")

Perennial = ("Sugarcane", "Citrus", "Grape",  "Cocoa", "Coffee",
             "Oil palm", "Date palm", "Others perennial")


In [4]:
# Load the datasets
# Root folder for saving rasters clipped by unitcode
root_folder = "../scratch"
shp_00 = "../data/admin_boundaries/merged_2000.shp"
shp_15 = "../data/admin_boundaries/merged_2015.shp"
scratch_folder = "../scratch"

# Load the crop and area data clipped by unit code 
raster_folder = "../data/Ras_shape_out/Rasters"
tiff_files = glob.glob("{}/**/*.tif".format(raster_folder), recursive=True)
tiff_files


# Folder with crop calendar data
CropCal_dir = "../data/CropCalender"

# Load crop calendar data and
# Rank each by crop area in each unit code
ranked_df_ir = {}
ranked_df_rf = {}
for year in ["00", "05", "10", "15"]:
  # Irrigated
  ranked_df_ir[year] =  rankbyarea(CropCal_dir = CropCal_dir,
                                  irrigated_rainfed = "ir", year=year)
  # Rainfed
  ranked_df_rf[year] = rankbyarea(CropCal_dir = CropCal_dir, 
                        irrigated_rainfed = "ir", year="00",
                         by_state = True)
    


# ranked_df_rf["15"]
# ranked_df_rf["00"]
ranked_df_ir["00"]

In [ ]:
# Clip the rasters 

root_folder_subfolders = {y:"{}/{}".format(root_folder, y) for y in ["00", "05", "10", "15"]}

for year in ["00", "05", "10", "15"]:
  shp_path = shp_15 #if year in ["15"] else shp_00
  raster_main_dir = "../data/cropRaster/{}{}".format("20", year)
  output_dir = "{}/Clipped".format(scratch_folder)

  raster_paths = glob.glob("{}/*.tif".format(raster_main_dir), recursive=False)
  for raster_path in raster_paths:
    mask_raster_by_geometries(shp_path, raster_path, scratch_folder, 
                              year,  visualize=False)

 





In [23]:


for year in ["00", "05", "10", "15"]:

  if ranked_df_ir[year] is None:
    print("ranked_df_ir {} is not available".format(year))
    continue
  # print(year)
  dist_array = {}
  irri_totals = {}

  # Get all unique UNIT CODEs
  basename_unit_codes = [os.path.splitext(os.path.basename(tiff_file))[0] \
                         for tiff_file in glob.glob("{}/*/**.tif".format(root_folder),
                                                    recursive=True)]
  unit_codes = np.unique(np.array(basename_unit_codes))

  for unit_code in unit_codes:
    print(unit_code)  

 
    # Get the crop-specific irrigated and rainfed growing area and crop calendar of the unit code district
    ranked_df_ir_unitcode = ranked_df_ir[year].loc[(ranked_df_ir[year]['unit_code'] == int(unit_code))]
    ranked_df_rf_unitcode = ranked_df_rf[year].loc[(ranked_df_rf[year]['unit_code'] == int(unit_code))]
    # print(ranked_df_unitcode)

    # SKIP UNIT CODES NOT IN IRRIGATED RANKED DATA
    if not ranked_df_ir_unitcode.shape[0]:
    # print("Unit code {} is not in ranked_df".format(unit_code))
      continue

    # LOAD THE AEI, ARA, AND CLE
    for aac in ["AEI", "Ara","CLE"]:
      tiff_file_aac = os.path.join("../{}".format(root_folder), "{}_{}".format(aac, year), "{}.tif".format(unit_code))
      # print(tiff_file_aac)
      dist_array["dist_array_{}".format(aac.lower())] = gdal.Open(tiff_file_aac, gdal.GA_ReadOnly).ReadAsArray()
 
     # Loop through the crops by rank while allocating area in the seven steps
    for j in range(1, int(np.max(ranked_df_ir_unitcode['rank']) + 1)):
      ranked_df_ir_rank = ranked_df_ir_unitcode.loc[(ranked_df_ir_unitcode['rank'] == j)]
      # ranked_df_rf_rank = ranked_df_rf_unitcode.loc[(ranked_df_rf_unitcode['rank'] == j)]
      # print(ranked_df_rank)
      # Extract array data for crop type and assign to corresponding variable in crop_dict
      crop_name = ranked_df_ir_rank.Crop.tolist()[0]


      tiff_file = getpath2croptiff(crop_name, root_folder, allcrops, year, unit_code)
      # SKIP THE ROW IF CROP NOT IN ALL CROPS
      if not tiff_file:
        print("Crop {} is not in allcrops dictionary".format(crop_name))
        continue
      # print(tiff_file)

    
      AH_array = gdal.Open(tiff_file, gdal.GA_ReadOnly).ReadAsArray()
      AH_array[AH_array < 0] = 0  

      
      # CALL THE STEPS ONE AT A TIME WITH THE DATA  AND FIX THE ERRORS
      step1 = step00(step=1, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     # ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = None, ATS_2 = None, AD_area = None, df_cumulative=None,
                     df_mon=None, TAD_12=None)
  
      # {"df_cumulative":df_cumulative, "AD_area" : AD_area1, "ATS_2": ATS_2, "df_tot":df_tot}
      step2 = step00(step=2, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     # ranked_df_rf_rank = ranked_df_rf_rank, 
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step1['df_tot'],  ATS_2 = step1['ATS_2'], AD_area = step1['AD_area'],
                     df_cumulative=step1['df_cumulative'], df_mon=step1['df_mon'],
                     c_s=step1['c_s'], r_s=step1['r_s'],
                     TAD_12=None)
    
      step3 = step00(step=3, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     # ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step2['df_tot'],  ATS_2 = step2['ATS_2'], AD_area = step2['AD_area'], 
                     df_cumulative=step2['df_cumulative'], df_mon=step2['df_mon'],
                     c_s=step2['c_s'], r_s=step2['r_s'],
                     TAD_12=None)


    # cHECK IF THE CROP IN BOTH IRRIGATED AND RAINFED
    
      step4 = step00(step=4, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     # ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step3['df_tot'],  ATS_2 = step3['ATS_2'], AD_area = step3['AD_area'],
                     df_cumulative=step3['df_cumulative'], df_mon=step3['df_mon'],
                     c_s=step3['c_s'], r_s=step3['r_s'],
                     TAD_12=None,
                     template_tiff= tiff_file)

      # if dist_array_area is not None and dist_array_aei is not None and dist_array_cle is not None:
    array0 = [dist_array["dist_array_ara"], dist_array["dist_array_aei"], dist_array["dist_array_cle"]]
    if any(element is None for element in array0):
      break

    # Skip unitcode if there are no crops
    if not ranked_df_ir_unitcode.shape[0]:
    # print("Unit code {} is not in ranked_df".format(unit_code))
      continue
    for j in range(1, int(np.max(ranked_df_rf_unitcode['rank']) + 1)):
      # ranked_df_ir_rank = ranked_df_ir_unitcode.loc[(ranked_df_ir_unitcode['rank'] == j)]
      ranked_df_rf_rank = ranked_df_rf_unitcode.loc[(ranked_df_rf_unitcode['rank'] == j)]
      
      # Extract array data for crop type and assign to corresponding variable in crop_dict
      crop_name = ranked_df_rf_rank.Crop.tolist()[0]
      tiff_file = getpath2croptiff(crop_name, root_folder, allcrops, year, unit_code)
      # SKIP THE ROW IF CROP NOT IN ALL CROPS
      if not tiff_file:
        print("Crop {} is not in allcrops dictionary".format(crop_name))
        continue
        
      step5 = step00(step=5, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step4['df_tot'],  ATS_2 = step4['ATS_2'], AD_area = step4['AD_area'],
                     df_cumulative=step4['df_cumulative'], df_mon=step4['df_mon'],
                     c_s=step4['c_s'], r_s=step4['r_s'],
                     TAD_12=None,
                     template_tiff= tiff_file)
        
      step6 = step00(step=6, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step5['df_tot'],  ATS_2 = step5['ATS_2'], AD_area = step5['AD_area'],
                     df_cumulative=step5['df_cumulative'], df_mon=step5['df_mon'],
                     c_s=step5['c_s'], r_s=step5['r_s'],
                     TAD_12=None,
                     template_tiff= tiff_file)
        
      step7 = step00(step=7, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step6['df_tot'],  ATS_2 = step6['ATS_2'], AD_area = step6['AD_area'],
                     df_cumulative=step6['df_cumulative'], df_mon=step6['df_mon'],
                     c_s=step6['c_s'], r_s=step6['r_s'],
                     TAD_12=None,
                     template_tiff= tiff_file)

  # print(step5)


00
ranked_df_ir 05 is not available
ranked_df_ir 10 is not available
ranked_df_ir 15 is not available


In [19]:
      step7 = step00(step=7, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step6['df_tot'],  ATS_2 = step6['ATS_2'], AD_area = step6['AD_area'],
                     df_cumulative=step6['df_cumulative'], df_mon=step6['df_mon'],
                     c_s=step6['c_s'], r_s=step6['r_s'],
                     TAD_12=None,
                     template_tiff= tiff_file)

TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union